In [3]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import faiss

# Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Create embeddings

documents = ["""
            company_name: Pii Holdings, Inc.
            datapoint_type: product
            datapoint_value: gas pipe fixtures
            source_excerpts: we do gas pipes on a daily basis.""",
            """
            company_name: XYZ Inc.
            datapoint_type: location
            datapoint_value: 3288 Main St. Buffalo NY
            source_excerpts: visit our website.""",
            """
            company_name: Pii Holdings, Inc.
            datapoint_type: location
            datapoint_value: 2150 Elmwood Avenue Buffalo, NY
            source_excerpts: they also have a division that designs and installs racks for solar panels.""",
            """
            company_name: Pii Holdings, Inc.
            datapoint_type: location
            datapoint_value: 2150 Elmwood Avenue Buffalo, NY
            source_excerpts: they also have a division that designs and installs racks for solar panels.""",
            """
            company_name: XYZ Inc.
            datapoint_type: product
            datapoint_value: 3D printing"""
]
doc_embeddings = embed_model.encode(documents)

# Step 2: Store embeddings in FAISS
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# Step 3: Query the database
query = "What company located in Buffalo sells gas line parts?"
query_embedding = embed_model.encode([query])
_, result_indices = index.search(query_embedding, k=1)

# Step 4: Retrieve and use the chunk
retrieved_chunk = documents[result_indices[0][0]]

# Use the retrieved chunk as context
input_text = f"Context: {retrieved_chunk}\nQuestion: {query}"

print(input_text)


Context: 
            company_name: XYZ Inc.
            datapoint_type: location
            datapoint_value: 3288 Main St. Buffalo NY
            source_excerpts: visit our website.
Question: What company located in Buffalo sells gas line parts?
